In [1]:
import pandas as pd
import pandas_datareader.data as web
import io
import requests
import time
import datetime

In [2]:
#function to get stock data
def yahoo_stocks(symbol, start, end):
    return web.DataReader(symbol, 'yahoo', start, end)

In [3]:
#get 7 year stock data for Apple
startDate = datetime.datetime(2010, 1, 4)
endDate = datetime.date.today()
stockData = yahoo_stocks('AAPL', startDate, endDate)

In [4]:
stockMarketData = yahoo_stocks('^GSPC', startDate, endDate)

In [5]:
stockMarketData.head(5)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-01-04,1116.560059,1133.869995,1116.560059,1132.989990,1132.989990,3991400000
2010-01-05,1132.660034,1136.630005,1129.660034,1136.520020,1136.520020,2491020000
2010-01-06,1135.709961,1139.189941,1133.949951,1137.140015,1137.140015,4972660000
2010-01-07,1136.270020,1142.459961,1131.319946,1141.689941,1141.689941,5270680000
2010-01-08,1140.520020,1145.390015,1136.219971,1144.979980,1144.979980,4389590000


In [6]:
#adding rows for missing dates
def add_missing_dates(dataframe, start, end):
    idx = pd.date_range(start, end)
    dataframe.index = pd.DatetimeIndex(dataframe.index)
    dataframe = dataframe.reindex(idx, fill_value='np.nan')
    return dataframe

In [7]:
stockData = add_missing_dates(stockData, startDate, endDate)

In [8]:
stockData.head(10)

,Open,High,Low,Close,Adj Close,Volume
2010-01-04,30.49,30.6429,30.34,30.5729,27.4065,123432400
2010-01-05,30.6571,30.7986,30.4643,30.6257,27.4539,150476200
2010-01-06,30.6257,30.7471,30.1071,30.1386,27.0172,138040000
2010-01-07,30.25,30.2857,29.8643,30.0829,26.9673,119282800
2010-01-08,30.0429,30.2857,29.8657,30.2829,27.1466,111902700
2010-01-09,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan
2010-01-10,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan
2010-01-11,30.4,30.4286,29.7786,30.0157,26.9071,115557400
2010-01-12,29.8843,29.9671,29.4886,29.6743,26.601,148614900
2010-01-13,29.6957,30.1329,29.1571,30.0929,26.9762,151473000


In [9]:
stockMarketData = add_missing_dates(stockMarketData, startDate, endDate)

In [10]:
stockMarketData.head(10)

,Open,High,Low,Close,Adj Close,Volume
2010-01-04,1116.56,1133.87,1116.56,1132.99,1132.99,3991400000
2010-01-05,1132.66,1136.63,1129.66,1136.52,1136.52,2491020000
2010-01-06,1135.71,1139.19,1133.95,1137.14,1137.14,4972660000
2010-01-07,1136.27,1142.46,1131.32,1141.69,1141.69,5270680000
2010-01-08,1140.52,1145.39,1136.22,1144.98,1144.98,4389590000
2010-01-09,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan
2010-01-10,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan
2010-01-11,1145.96,1149.74,1142.02,1146.98,1146.98,4255780000
2010-01-12,1143.81,1143.81,1131.77,1136.22,1136.22,4716160000
2010-01-13,1137.31,1148.4,1133.18,1145.68,1145.68,4170360000


In [11]:
#convert the columns to numeric
def convert_to_numeric(dataframe):
    for col in dataframe:
        dataframe[col] = pd.to_numeric(dataframe[col], errors='coerce')
    return dataframe

In [12]:
stockDataNumeric = convert_to_numeric(stockData)

In [13]:
stockMarketDataNumeric = convert_to_numeric(stockMarketData)

In [14]:
def interpolate(dataframe):
    features = list(dataframe)
    for feature in features:
        dataframe[feature] = dataframe[feature].interpolate()
    return dataframe

In [15]:
stockDataInterpolated = interpolate(stockDataNumeric)

In [16]:
stockDataInterpolated.head(10)

,Open,High,Low,Close,Adj Close,Volume
2010-01-04,30.490000,30.642857,30.340000,30.572857,27.406532,1.234324e+08
2010-01-05,30.657143,30.798571,30.464285,30.625713,27.453915,1.504762e+08
2010-01-06,30.625713,30.747143,30.107143,30.138571,27.017223,1.380400e+08
2010-01-07,30.250000,30.285715,29.864286,30.082857,26.967278,1.192828e+08
2010-01-08,30.042856,30.285715,29.865715,30.282858,27.146566,1.119027e+08
2010-01-09,30.161904,30.333334,29.836667,30.193810,27.066742,1.131209e+08
2010-01-10,30.280952,30.380953,29.807620,30.104763,26.986917,1.143392e+08
2010-01-11,30.400000,30.428572,29.778572,30.015715,26.907093,1.155574e+08
2010-01-12,29.884285,29.967142,29.488571,29.674286,26.601023,1.486149e+08
2010-01-13,29.695715,30.132856,29.157143,30.092857,26.976244,1.514730e+08


In [17]:
stockMarketDataInterpolated = interpolate(stockMarketDataNumeric)

In [18]:
stockMarketDataInterpolated.head(10)

,Open,High,Low,Close,Adj Close,Volume
2010-01-04,1116.560059,1133.869995,1116.560059,1132.989990,1132.989990,3.991400e+09
2010-01-05,1132.660034,1136.630005,1129.660034,1136.520020,1136.520020,2.491020e+09
2010-01-06,1135.709961,1139.189941,1133.949951,1137.140015,1137.140015,4.972660e+09
2010-01-07,1136.270020,1142.459961,1131.319946,1141.689941,1141.689941,5.270680e+09
2010-01-08,1140.520020,1145.390015,1136.219971,1144.979980,1144.979980,4.389590e+09
2010-01-09,1142.333334,1146.840007,1138.153321,1145.646647,1145.646647,4.344987e+09
2010-01-10,1144.146647,1148.289998,1140.086670,1146.313313,1146.313313,4.300383e+09
2010-01-11,1145.959961,1149.739990,1142.020020,1146.979980,1146.979980,4.255780e+09
2010-01-12,1143.810059,1143.810059,1131.770020,1136.219971,1136.219971,4.716160e+09
2010-01-13,1137.310059,1148.400024,1133.180054,1145.680054,1145.680054,4.170360e+09


In [19]:
#difference between the previous day and todays closing value
def prev_diff(dataframe):
    close = dataframe['Close']
    prev_diff = [0]
    for i in range(1, len(dataframe)):
        prev_diff.append(round((close[i]-close[i-1]),6))
    return prev_diff

In [20]:
stockDataInterpolated['prev_diff'] = prev_diff(stockDataInterpolated)

In [21]:
stockMarketDataInterpolated['sm_prev_diff'] = prev_diff(stockMarketDataInterpolated)

In [22]:
stockDataPrevAdd = stockDataInterpolated

In [23]:
stockMarketDataPrevAdd = stockMarketDataInterpolated

In [24]:
#convert pandas dataframe to StockDataFrame
from stockstats import StockDataFrame 

stockstats_df = StockDataFrame.retype(stockDataPrevAdd)

In [25]:
stockstats_df.head(5)

,open,high,low,close,adj close,volume,prev_diff
2010-01-04,30.490000,30.642857,30.340000,30.572857,27.406532,123432400.0,0.000000
2010-01-05,30.657143,30.798571,30.464285,30.625713,27.453915,150476200.0,0.052856
2010-01-06,30.625713,30.747143,30.107143,30.138571,27.017223,138040000.0,-0.487142
2010-01-07,30.250000,30.285715,29.864286,30.082857,26.967278,119282800.0,-0.055714
2010-01-08,30.042856,30.285715,29.865715,30.282858,27.146566,111902700.0,0.200001


In [26]:
#50 day moving average
stockstats_df['close_50_sma']

2010-01-04     30.572857
2010-01-05     30.599285
2010-01-06     30.445714
2010-01-07     30.355000
2010-01-08     30.340571
2010-01-09     30.316111
2010-01-10     30.285918
2010-01-11     30.252143
2010-01-12     30.187937
2010-01-13     30.178429
2010-01-14     30.154805
2010-01-15     30.093452
2010-01-16     30.066566
2010-01-17     30.066760
2010-01-18     30.088619
2010-01-19     30.128080
2010-01-20     30.135084
2010-01-21     30.112262
2010-01-22     30.014248
2010-01-23     29.938702
2010-01-24     29.882415
2010-01-25     29.842760
2010-01-26     29.824379
2010-01-27     29.819077
2010-01-28     29.765114
2010-01-29     29.675577
2010-01-30     29.597381
2010-01-31     29.529311
2010-02-01     29.470320
2010-02-02     29.420643
                 ...    
2017-10-18    157.592500
2017-10-19    157.445100
2017-10-20    157.290100
2017-10-21    157.133567
2017-10-22    156.986350
2017-10-23    156.848450
2017-10-24    156.739000
2017-10-25    156.625600
2017-10-26    156.535600


In [27]:
stockstats_df.head(5)

,open,high,low,close,adj close,volume,prev_diff,close_50_sma
2010-01-04,30.490000,30.642857,30.340000,30.572857,27.406532,123432400.0,0.000000,30.572857
2010-01-05,30.657143,30.798571,30.464285,30.625713,27.453915,150476200.0,0.052856,30.599285
2010-01-06,30.625713,30.747143,30.107143,30.138571,27.017223,138040000.0,-0.487142,30.445714
2010-01-07,30.250000,30.285715,29.864286,30.082857,26.967278,119282800.0,-0.055714,30.355000
2010-01-08,30.042856,30.285715,29.865715,30.282858,27.146566,111902700.0,0.200001,30.340571


In [28]:
stockMarketStats_df = StockDataFrame.retype(stockMarketDataPrevAdd)

In [29]:
stockMarketStats_df['close_50_sma']

2010-01-04    1132.989990
2010-01-05    1134.755005
2010-01-06    1135.550008
2010-01-07    1137.084991
2010-01-08    1138.663989
2010-01-09    1139.827765
2010-01-10    1140.754272
2010-01-11    1141.532486
2010-01-12    1140.942206
2010-01-13    1141.415991
2010-01-14    1142.056352
2010-01-15    1141.554158
2010-01-16    1141.402301
2010-01-17    1141.525709
2010-01-18    1141.869328
2010-01-19    1142.391868
2010-01-20    1142.135878
2010-01-21    1140.710551
2010-01-22    1138.134207
2010-01-23    1135.899164
2010-01-24    1133.956665
2010-01-25    1132.266818
2010-01-26    1130.523480
2010-01-27    1129.147501
2010-01-28    1127.362803
2010-01-29    1125.305387
2010-01-30    1123.589508
2010-01-31    1122.178571
2010-02-01    1121.041032
2010-02-02    1120.450329
                 ...     
2017-10-18    2510.586824
2017-10-19    2512.677024
2017-10-20    2514.748225
2017-10-21    2516.653223
2017-10-22    2518.583522
2017-10-23    2520.539120
2017-10-24    2522.671216
2017-10-25  

In [30]:
stockMarketStats_df.head(5)

,open,high,low,close,adj close,volume,sm_prev_diff,close_50_sma
2010-01-04,1116.560059,1133.869995,1116.560059,1132.989990,1132.989990,3.991400e+09,0.000000,1132.989990
2010-01-05,1132.660034,1136.630005,1129.660034,1136.520020,1136.520020,2.491020e+09,3.530030,1134.755005
2010-01-06,1135.709961,1139.189941,1133.949951,1137.140015,1137.140015,4.972660e+09,0.619995,1135.550008
2010-01-07,1136.270020,1142.459961,1131.319946,1141.689941,1141.689941,5.270680e+09,4.549926,1137.084991
2010-01-08,1140.520020,1145.390015,1136.219971,1144.979980,1144.979980,4.389590e+09,3.290039,1138.663989


In [31]:
#convery StockDataFrame to pandas DataFrame 
stockDataNew = pd.DataFrame(stockstats_df)

In [32]:
stockMarketDataNew = pd.DataFrame(stockMarketStats_df)

In [33]:
stockMarketDataNew.head(5)

,open,high,low,close,adj close,volume,sm_prev_diff,close_50_sma
2010-01-04,1116.560059,1133.869995,1116.560059,1132.989990,1132.989990,3.991400e+09,0.000000,1132.989990
2010-01-05,1132.660034,1136.630005,1129.660034,1136.520020,1136.520020,2.491020e+09,3.530030,1134.755005
2010-01-06,1135.709961,1139.189941,1133.949951,1137.140015,1137.140015,4.972660e+09,0.619995,1135.550008
2010-01-07,1136.270020,1142.459961,1131.319946,1141.689941,1141.689941,5.270680e+09,4.549926,1137.084991
2010-01-08,1140.520020,1145.390015,1136.219971,1144.979980,1144.979980,4.389590e+09,3.290039,1138.663989


In [40]:
stockMarketDataNew.columns = ['open','high','low', 'close', 'adj close', 'volume', 'sm_prev_diff', 'sm_close_50_sma']

In [41]:
stockMarketDataNew.head(5)

,open,high,low,close,adj close,volume,sm_prev_diff,sm_close_50_sma
2010-01-04,1116.560059,1133.869995,1116.560059,1132.989990,1132.989990,3.991400e+09,0.000000,1132.989990
2010-01-05,1132.660034,1136.630005,1129.660034,1136.520020,1136.520020,2.491020e+09,3.530030,1134.755005
2010-01-06,1135.709961,1139.189941,1133.949951,1137.140015,1137.140015,4.972660e+09,0.619995,1135.550008
2010-01-07,1136.270020,1142.459961,1131.319946,1141.689941,1141.689941,5.270680e+09,4.549926,1137.084991
2010-01-08,1140.520020,1145.390015,1136.219971,1144.979980,1144.979980,4.389590e+09,3.290039,1138.663989


In [42]:
#10 day volatility
import math

def ten_day_volatility(dataframe):
    volatility = dataframe['close'].rolling(window=10,center=False).std(ddof=0)
    # daily_pct_change = stockstats_df['close'] / stockstats_df['close'].shift(1) - 1
    # volatility2 = daily_pct_change.rolling(window=10,center=False).std(ddof=0) * math.sqrt(10)
    top = dataframe[0:9]
    top_vol = top['close'].rolling(window=2,center=False).std(ddof=0)
    top_vol[0] = 0
    volatility[0:9] = top_vol
    return volatility

In [43]:
volatility = ten_day_volatility(stockDataNew)
stockDataNew['10_day_volatility'] = volatility

In [44]:
stockDataNew.head(5)

,open,high,low,close,adj close,volume,prev_diff,close_50_sma,10_day_volatility
2010-01-04,30.490000,30.642857,30.340000,30.572857,27.406532,123432400.0,0.000000,30.572857,0.000000
2010-01-05,30.657143,30.798571,30.464285,30.625713,27.453915,150476200.0,0.052856,30.599285,0.026428
2010-01-06,30.625713,30.747143,30.107143,30.138571,27.017223,138040000.0,-0.487142,30.445714,0.243571
2010-01-07,30.250000,30.285715,29.864286,30.082857,26.967278,119282800.0,-0.055714,30.355000,0.027857
2010-01-08,30.042856,30.285715,29.865715,30.282858,27.146566,111902700.0,0.200001,30.340571,0.100001


In [45]:
volatility = ten_day_volatility(stockMarketDataNew)
stockMarketDataNew['sm_10_day_volatility'] = volatility

In [46]:
stockMarketDataNew.head(5)

,open,high,low,close,adj close,volume,sm_prev_diff,sm_close_50_sma,sm_10_day_volatility
2010-01-04,1116.560059,1133.869995,1116.560059,1132.989990,1132.989990,3.991400e+09,0.000000,1132.989990,0.000000
2010-01-05,1132.660034,1136.630005,1129.660034,1136.520020,1136.520020,2.491020e+09,3.530030,1134.755005,1.765015
2010-01-06,1135.709961,1139.189941,1133.949951,1137.140015,1137.140015,4.972660e+09,0.619995,1135.550008,0.309998
2010-01-07,1136.270020,1142.459961,1131.319946,1141.689941,1141.689941,5.270680e+09,4.549926,1137.084991,2.274963
2010-01-08,1140.520020,1145.390015,1136.219971,1144.979980,1144.979980,4.389590e+09,3.290039,1138.663989,1.645019


In [52]:
stockMarketDataNew.columns = ['sm_open', 'sm_high', 'sm_low', 'sm_close', 'sm_adj_close', 'sm_volume', 'sm_prev_diff', 'sm_close_50_sma', 'sm_10_day_volatility']

In [53]:
finalData = pd.concat([stockDataNew, stockMarketDataNew], axis=1)

In [54]:
finalData.head(5)

,open,high,low,close,adj close,volume,prev_diff,close_50_sma,10_day_volatility,sm_open,sm_high,sm_low,sm_close,sm_adj_close,sm_volume,sm_prev_diff,sm_close_50_sma,sm_10_day_volatility
2010-01-04,30.490000,30.642857,30.340000,30.572857,27.406532,123432400.0,0.000000,30.572857,0.000000,1116.560059,1133.869995,1116.560059,1132.989990,1132.989990,3.991400e+09,0.000000,1132.989990,0.000000
2010-01-05,30.657143,30.798571,30.464285,30.625713,27.453915,150476200.0,0.052856,30.599285,0.026428,1132.660034,1136.630005,1129.660034,1136.520020,1136.520020,2.491020e+09,3.530030,1134.755005,1.765015
2010-01-06,30.625713,30.747143,30.107143,30.138571,27.017223,138040000.0,-0.487142,30.445714,0.243571,1135.709961,1139.189941,1133.949951,1137.140015,1137.140015,4.972660e+09,0.619995,1135.550008,0.309998
2010-01-07,30.250000,30.285715,29.864286,30.082857,26.967278,119282800.0,-0.055714,30.355000,0.027857,1136.270020,1142.459961,1131.319946,1141.689941,1141.689941,5.270680e+09,4.549926,1137.084991,2.274963
2010-01-08,30.042856,30.285715,29.865715,30.282858,27.146566,111902700.0,0.200001,30.340571,0.100001,1140.520020,1145.390015,1136.219971,1144.979980,1144.979980,4.389590e+09,3.290039,1138.663989,1.645019


In [59]:
from sklearn import preprocessing

finalDataNorm = preprocessing.normalize(finalData, norm='l2', axis=1)

In [62]:
len(finalDataNorm[0])

18